In [1]:
from typing import Annotated 

from fastapi import Depends, FastAPI 

app = FastAPI() 



In [2]:

async def common_parameters(q: str | None = None, skip: int = 0, limit: int = 100): 
    return {"q": q, "skip": skip, "limit": limit} 

@app.get("/items/") 
async def read_items(commons: Annotated[dict, Depends(common_parameters)]): 
    return commons 

@app.get("/users/") 
async def get_users(commons: Annotated[dict, Depends(common_parameters)]): 
    return commons


remove code duplication 

In [4]:
async def common_parameters(q: str | None = None, skip: int = 0, limit: int = 100): 
    return {"q": q, "skip": skip, "limit": limit} 

CommonsDep = Annotated[dict, Depends(common_parameters)]

@app.get("/items/") 
async def read_items(commons: CommonsDep): 
    return commons 

@app.get("/users/") 
async def get_users(commons: CommonsDep): 
    return commons

# Classes as Dependencies

In [6]:
from typing import Annotated

from fastapi import Depends, FastAPI

app = FastAPI()


fake_items_db = [{"item_name": "Foo"}, {"item_name": "Bar"}, {"item_name": "Baz"}]


class CommonQueryParams:
    def __init__(self, q: str | None = None, skip: int = 0, limit: int = 100):
        self.q = q
        self.skip = skip
        self.limit = limit


@app.get("/items/")
async def read_items(commons: Annotated[CommonQueryParams, Depends(CommonQueryParams)]):
    response = {}
    if commons.q:
        response.update({"q": commons.q})
    items = fake_items_db[commons.skip : commons.skip + commons.limit]
    response.update({"items": items})
    return response

# Sub-dependencies

In [7]:
from typing import Annotated

from fastapi import Cookie, Depends, FastAPI

app = FastAPI()

In [8]:
def query_extractor(q: str | None = None): 
    return q 

def query_or_string_extractor(
    q: Annotated[str, Depends(query_extractor)], 
    last_query: Annotated[str | None, Cookie()] = None,
):
    if not q: 
        return last_query 
    return q 

In [9]:
@app.get("/items/")
async def read_query(
    query_or_default: Annotated[str, Depends(query_or_string_extractor)],
):
    return {"q_or_cookie": query_or_default}

## Using the same dependency multiple times 

- If one of your dependencies is declared multiple times for the same path operation, for example, multiple dependencies have a common sub-dependency, **FastAPI will know to call that sub-dependency only once per request.**


- Dependencies called only once per request, even if used multiple times
- Returned value is cached and reused for all dependants in the same request
- Prevents redundant calls to the same dependency

## **When to USE Cache (Default Behavior)**

### **Database Connections**
```python
async def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

# Same DB session reused throughout request
```

### **User Authentication**
```python
async def get_current_user(token: str = Depends(oauth2_scheme)):
    user = decode_token(token)
    return user

# User validated once, reused in multiple places
```

### **Configuration/Settings**
```python
def get_settings():
    return Settings()  # Expensive config loading

# Load settings once per request
```

### **External API Calls**
```python
async def get_exchange_rate():
    response = await fetch_external_api()
    return response.rate

# Avoid multiple API calls in same request
```

### **Complex Calculations**
```python
def calculate_user_permissions(user: User = Depends(get_current_user)):
    # Heavy permission calculation logic
    return permissions

# Calculate once, use everywhere
```

## **When to DISABLE Cache (`use_cache=False`)**

### **Random/Unique Value Generation**
```python
def generate_request_id():
    return str(uuid.uuid4())

# Need different ID each time it's called
router.get("/", dependencies=[Depends(generate_request_id, use_cache=False)])
```

### **Time-Sensitive Operations**
```python
def get_current_timestamp():
    return datetime.now()

# Need fresh timestamp each call
Depends(get_current_timestamp, use_cache=False)
```

### **Counters/Metrics**
```python
def increment_counter():
    metrics.counter += 1
    return metrics.counter

# Must increment each time
Depends(increment_counter, use_cache=False)
```

### **Side Effects Required**
```python
def log_access():
    logger.info(f"Access at {datetime.now()}")
    # Need to log each access point

Depends(log_access, use_cache=False)
```

### **Fresh Data Reads**
```python
def get_latest_stock_price():
    # Real-time data that changes rapidly
    return fetch_current_price()

Depends(get_latest_stock_price, use_cache=False)
```

### **Resource Cleanup/Multiple Instances**
```python
def get_temp_file():
    file = create_temp_file()
    yield file
    file.cleanup()  # Need separate cleanup for each usage

Depends(get_temp_file, use_cache=False)
```

## **Quick Decision Guide**

**Use Cache When:**
- Same result expected throughout request
- Expensive operations (DB, API, calculations)
- Stateless operations
- Authentication/authorization

**Disable Cache When:**
- Need different values each call
- Side effects are important
- Time-sensitive data
- Generating unique identifiers
- Tracking/counting operations

In [ ]:
async def needy_dependency(fresh_value: Annotated[str, Depends(get_value, use_cache=False)]):
    return {"fresh_value": fresh_value}

# Dependencies in path operation decorators 

You can add dependencies that don't need to return values to the path operation decorator's dependencies parameter as a list instead of declaring them as function parameters with `Depends`.

Cases where you DON'T need the return value:

- Authentication/authorization checks that just verify permissions without returning user data
- Logging or audit trail dependencies that record actions but don't provide data to the function
- Rate limiting that blocks requests but doesn't return limit information
- Database connection setup that establishes connections but you use a global connection object
- Cache warming or cleanup operations that perform side effects only

Cases where you DO need the return value:

- Getting the current authenticated user object to use in your function logic
- Database sessions that need to be passed to repository functions
- Configuration objects that contain settings your function needs
- Parsed and validated request data (like JSON payloads or query parameters)
- Computed values or processed data that your function logic depends on


In [4]:
# Authentication/authorization checks that just verify permissions without returning user data

from typing import Annotated 
from fastapi import Depends, FastAPI, Header, HTTPException 

app = FastAPI()


async def verify_token(x_token: Annotated[str, Header()]):
    if x_token != "fake-super-secret-token":
        raise HTTPException(status_code=400, detail="X-Token header invalid")


async def verify_key(x_key: Annotated[str, Header()]):
    if x_key != "fake-super-secret-key":
        raise HTTPException(status_code=400, detail="X-Key header invalid")
    return x_key

@app.get("/items/", dependencies=[Depends(verify_token), Depends(verify_key)])
async def read_items(): 
    return [{"item": "Foo"}, {"item": "Bar"}]

# Global Dependencies
For some types of applications you might want to add dependencies to the whole application.

Similar to the way you can add dependencies to the path operation decorators, you can add them to the FastAPI application.

In that case, they will be applied to all the path operations in the application

**Cases where you SHOULD add global dependencies:**
- Authentication middleware that needs to run on every endpoint
- Request logging/audit trails for all API calls
- CORS headers that apply to all routes
- Rate limiting across the entire application
- Database connection initialization for all endpoints
- Security headers (CSP, HSTS) that should be on every response
- Request ID generation for tracing across all operations

**Cases where you should NOT add global dependencies:**
- Role-based authorization that only applies to specific endpoints (admin routes vs public routes)
- Input validation that's specific to certain endpoints
- Caching that only makes sense for read operations, not write operations
- File upload handling that only applies to specific endpoints
- Payment processing logic that only applies to billing endpoints
- Third-party API integrations that only certain endpoints use
- Heavy computations that would slow down simple endpoints unnecessarily

In [6]:
from fastapi import Depends, FastAPI, Header, HTTPException
from typing_extensions import Annotated


async def verify_token(x_token: Annotated[str, Header()]):
    if x_token != "fake-super-secret-token":
        raise HTTPException(status_code=400, detail="X-Token header invalid")


async def verify_key(x_key: Annotated[str, Header()]):
    if x_key != "fake-super-secret-key":
        raise HTTPException(status_code=400, detail="X-Key header invalid")
    return x_key

# global
app = FastAPI(dependencies=[Depends(verify_token), Depends(verify_key)])


@app.get("/items/")
async def read_items():
    return [{"item": "Portal Gun"}, {"item": "Plumbus"}]


@app.get("/users/")
async def read_users():
    return [{"username": "Rick"}, {"username": "Morty"}]

# Dependencies with yield 

FastAPI supports dependencies that **do some extra steps after finishing**.

To do this, use yield instead of return, and write the extra steps (code) after.

- A database dependency with yield
- A dependency with yield and try
- Sub-dependencies with yield
- Dependencies with yield and HTTPException
- Dependencies with yield and except
- Always raise in Dependencies with yield and except
- Execution of dependencies with yield
- Dependencies with yield, HTTPException, except and Background Tasks
- Dependencies with yield and except, Technical Details
- Background Tasks and Dependencies with yield, Technical Details
- Context Managers
- What are "Context Managers"
- Using context managers in dependencies with yield

In [ ]:
async def get_db():
    db = DBSession()
    try:
        yield db
    finally:
        db.close()